<a href="https://colab.research.google.com/github/ssalazara/MACI_Analitica1/blob/main/Analitica_Tarea1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tarea 1. 

## Web scraping, Manejo de datos y Clasificación.


**Ayudante:** Martina Cádiz (mcadiz2018@inf.udec.cl)

---


La tarea debe ser subida a la plataforma en un archivo con el formato **NombreApellido_tarea1.zip**. Este debe contener el código solicitado, junto con el conjunto de datos utilizado. 





---
---

## 1. Web scraping (47 ptos)

Una emprendedora ha comenzado su propia empresa de Notebooks y quieren dar una dura pelea a las grandes empresas como Apple, Asus, HP, etc. Para esto contrataron a una empresa consultora quien les realizo un estudio de mercado, técnico y financiero que les permitió reunir información relevante sobre la industria, el mercado, los costos de los materiales y cómo seria su rentabilidad de aquí a un cierto horizonte de tiempo. Sin embargo, a ellos también les gustaría tener información actualizada, en todo momento, de los precios de ventas de sus competidores, así como también, un análisis actual de los precios a nivel de tienda y marca. Ella no tiene conocimientos de Análisis de Datos y de Machine Learning, por ende, lo/la contrata a usted para que realice este proyecto.

En primer lugar, ella le comenta que no cuenta con una base de datos disponible para realizar algún
análisis y que le gustaría obtener información actualizada del precio al cual están vendiendo los
productos sus competidores. Además, otro de los problemas iniciales que surgieron es que no sabe
cómo estimar el precio de los móviles que fabricará su empresa, y que no le gustaría simplemente
asumir cosas. Por lo tanto, ella le solicita a usted:

1. Que genere un código que permita extraer la información de Notebooks de la página www.solotodo.cl/notebooks, ya que recopila información de varias tiendas y marcas diferentes y no tiene restricciones de extracción de datos mediante técnicas de scrapping. Debe extraer hasta la **página 90**.

2. Que averigüe alguna relación entre las funciones de los Notebooks y su precio de venta. (Luego de recopilar los datos de ventas).



### Los datos que le interesa a la emprendedora son los siguientes:

**Información que se puede extraer desde la página general:**

- Nombre Notebook (id)
- Marca Notebook

*Procesador:*
- Modelo procesador

*RAM:*
- GB
       
*Almacenamiento: (se pedirá extraer el tipo de memoria y sus GB, en caso de que tenga más de una se debe seleccionar la primera)* 
- Tipo de disco duro 
- GB
    
*Tarjeta de video:*
- Marca GPU dedicada
- GB dedicada
    
**Información que deberán extraer dentro de cada producto:**
- Tabla de precios de los computadores en las diferentes tiendas

Como habrán aprendido en clases, en el proceso de recolección de datos no siempre encontrarán toda la información en cada uno de los productos, por lo cual deberán decidir en cómo trabajar con estos tipos de datos faltantes. Para esto tendrán que explicar cada decisión tomada para la representación de estos. Para efectos de revisión, todos los datos obtenidos deberán ser guardados en un archivo ".pkl" (pickle), ya que la página es actualizada constantemente.

**Resumen:** 

En la primera parte se espera que: 
1. Extraigan la información **(15 pts)**,
2. La almacenen en un DataFrame (información y precios) **(2 pts)**,
3. Que limpien los datos **(7 pts)**, 
4. Generen las variables de interes correspondientes (columnas) **(7 pts)**,
5. Que expliquen las decisiones tomadas y guarden los datos trabajados en un archivo “.pkl” **(3 pts)**,
6. Y que respondan preguntas relacionadas a los datos **(13 pts)**.

En la segunda parte se evaluará que:
1. Preprocesen los datos **(3 pts)**,
2. Dividan los conjuntos de datos **(3 pts)**,
3. Entrenen algún algoritmo sobre los datos de entrenamiento **(5 pts)**,
4. Validen sus resultados con el uso de validación cruzada **(5 pts)**,
5. Realicen un testeo **(2 pts)**,
6. Por último que extraigan conclusiones **(5 pts)**.



### Librerías

In [1]:
# Recuerde utilizar el chromedriver.exe de acuerdo a la explicación que se dio en la práctica para que no tenga problemas
!pip install kora -q
!pip install requests
!pip install bs4

     |████████████████████████████████| 57 kB 973 kB/s 
     |████████████████████████████████| 1.6 MB 7.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Librerías

# Web Scraping
from kora.selenium import wd
import requests
import bs4

# Data handling
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re

# Computation
import time as tm
from time import time

# Folder related
import os
!mkdir data 

# Guardar archivos
import pickle

# Warnings
import warnings
warnings.filterwarnings(action = "ignore")



/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


### Extracción de información (15 pts)

In [3]:
# Sincronización con drive para la gestión de archivos

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Creación de listas que contendrán las observaciones por cada notebook

# URL de cada notebook, desde donde podemos extraer el ID
url_product = []

# Nombre del notebook
names = []

# Modelo de procesador
processor = []

# RAM del notebook en GB
ram = []

# Almacenamiento
storage = []

# Modelo de trajeta de video
video = []

Utilización de BS4 para la extracción de los datos

In [7]:
#url página solotodo.cl

url_general = 'https://www.solotodo.cl'
type_technology='notebooks'

#ciclo para extraer la información de las páginas y llenar las listas anteriores

for i in range (1, 3):
  page_param = {'ordering': 'offer_price_usd', 'page': i}
  res = requests.get('{}/{}'.format(url_general, type_technology), params=page_param)
 
  soup = bs4.BeautifulSoup(res.text, 'lxml')
  links = soup.find_all('div',  attrs = {'class' :"d-flex flex-column category-browse-result"})

  for link in links:
    h3 = link.find('h3')
    dl = link.find('dl')
    dd = dl.find_all('dd')

    url_product.append(url_general + str(h3.a['href'].strip()))
    names.append(h3.a.text.strip())
    processor.append(str(dd[0].text.strip()).replace('\r\n\t\t', ''))
    ram.append(str(dd[1].text.strip()))
    storage.append(str(dd[3].text.strip()))
    video.append(str(dd[4].text.strip()))


Dataframe obtenido con el rango de páginas seleccionadas

### Almacenamiento en DataFrame (2 pts)

In [ ]:
Y# Seleccione el path donde quiera guardar la data
path_save_data = './data'

if not os.path.isfile('{}/data_without_clean.pkl'.format(path_save_data)): 
    df_inicial = pd.DataFrame(#crear dataframe)
    df_inicial.to_pickle('{}/data_without_clean.pkl'.format(path_save_data))
    
# Data guardada
else: df_inicial = pd.read_pickle('data/data_without_clean.pkl')
    
df_inicial

### Limpieza de datos y generación columnas de interés (17 pts)

**Limpieza general de los datos**

**Generamos columnas de interes**

**Exploración de los datos "limpios"**

**Falta limpiar las marcas de las tiendas.**

En este caso no trabajaremos con los precios de las tiendas que incluyan accesorios, ya que **pueden sesgar el precio de los Notebooks** propiamente tal. Para esto **eliminarmos las filas que contengan tiendas con la palabra "Incluye"**.

**Se guardan los datos limpios y extraídos en un csv.**

In [ ]:
#guarda el dataframe en un archivo pickle
df_final_clean.to_pickle('data/data_clean_notebooks.pkl')

## 2. Manejo de datos (13 ptos)

En primera instancia a la emprendedora le gustaría un análisis de filtrado de los Notebooks a nivel de tienda y marca. Para esto le piden a usted que responda las siguientes preguntas:

### Pregunta 2.1 (3 ptos):

Entregue una tabla:

*   Notebooks que tengan el precio normal entre mínimo $400.000$ mil y máximo $800.000$ mil, que sean Intel Core i7 y que tengan 16 o menos GB de RAM.



### Pregunta 2.2 (2 pts - bonus): 

* ¿Cuáles marcas y tiendas venden Notebooks en el mercado chileno de acuerdo con solotodo?
* ¿Cuántas marcas y tiendas venden Notebooks en el mercado chileno de acuerdo con solotodo?

Utilice la primera palabra del nombre del Notebook como la marca.

### Pregunta 2.3 (2 pts - bonus):

¿Cuál es la cantidad de Notebooks que se venden por marca (considere todos los Notebooks, independiente si se vende en otra tienda o no). Para esto se le pide la siguiente tabla:

| Marca | Cantidad |
|:-----------:|:------:|
| Asus	 |38
| Apple | 286 |
| HP | 462 |
| ... | ... |

Ademas, con esta información construya un gráfico de barras. Para esto pueden utilizar la librería **Matplotlib**. 

**Reminder:** Un gráfico siempre debe tener título y labels en el eje x e y.

### Pregunta 2.4 (6 pts - bonus):

- Muestre en una tabla los precios normales promedios de todas las marcas (redondeé a dos decimales) y luego las estadísticas descriptivas (desviación estándar, promedio, valor máximo y mínimo) de las 3 marcas de Notebooks que tengan el mayor precio normal promedio y las 2 marcas que tengan el menor precio normal promedio (considere todos los Notebooks).

    *Ejemplo (primera tabla).*

| Marca | Precio normal promedio |
|:----------------:|:----------------:|
|  Marca 1 | 4.100.000 |
| ... | ... |
| Marca 26 | 250.000 |

- Entregue una tabla con los 30 Notebooks con mayor descuento (precio normal - precio oferta). La tabla debe contener los descuentos de mayor a menor y sin repetición de un mismo notebook en diferentes tiendas (mantenga el Notebook de una misma tienda con el descuento mayor).

    *Ejemplo.*

| Tienda | Nombre | Precio normal | Precio oferta | Descuento |
|:----------------:|:----------------:|:----------------:|:----------------:|:----------------:|
| Tienda x |  Nombre 1 | 900.000 |  830.000 | 70.000 |
| ... | ... | ... | ... | ... |
| Tienda x | Nombre 30 | 643.000 | 603.000 | 40.000 |



## 3. Clasificación (23 ptos)

Para resolver el segundo problema (encontrar una relación entre las funciones de un Notebook
y su precio de venta), en primera instancia, se le ocurre formular un problema de clasificación en
vez de ir directo a uno de regresión. Es decir, se le ocurre formular un modelo que permita clasificar
un celular entre ciertos rangos de precios de acuerdo con las características que este tiene. Los
rangos de precios que usted debe generar para predecir (etiquetas) son los siguientes:

| Etiqueta | Precio | Rango |
|:----------------:|:----------------:|:----------------:|
| 0 | Bajo | [0 – 500.000) |
| 1 | Medio | [500.000 – 700.000) |
| 2 | Alto | [700.000 – más) |


**Consideraciones:**
- En esta tarea no tiene que predecir el precio real, sino más bien, debe
clasificar en qué rango de precios estará un celular de acuerdo con sus características.
- Revise que las variables de una misma columna estén en la misma unidad de medida.
- Para la clasificación usted puede utilizar las carcterísticas que estime conveniente, el único requisito es justificar sus decisiones. 
- Siguiendo la misma lógica del punto anterior, puede eliminar filas si aclara y menciona su eliminación.


### Preparación datos

Si dentro de esta etapa toma ciertas decisiones sobre el conjunto de datos, dejelas por escrita.

#### 1.1 Preprocese los datos para entrenar un algoritmo de Machine Learning. (3 pts)

Debe crear las etiquetas mencionadas en el enunciado, para esto se recomienda el uso de filtros para la creación de las etiquetas.

#### 1.2 Divida el conjunto de datos en entrenamiento, validación y test. (3 pts)

### Modelamiento

Se espera que entrene el modelo con los datos de entrenamiento y busqué los mejores hiperparametros sobre el conjunto de validación. Recuerde utilizar las métricas vistas en clases.

#### 1.3 Entrene el algoritmo sobre el conjunto de entrenamiento. (5 pts)

#### 1.4 Valide sus resultados. (5 pts)

### Modelo en producción (7 pts)

Luego de asegurarse de que encontró los mejores hiperparametros para el modelo, debe ponerlo en producción. Para esto se utilizaran los datos de test (es muy importante que estos datos no hayan sido utilizados antes), ya que esto permite evaluar el desempeño de su modelo sobre datos que no se tienen disponibles en la práctica.

#### 1.5 Pruebe el algoritmo sobre datos con los que no ha trabajado antes (datos de test). (2 pts)

#### 1.6 Concluya los resultados obtenidos, tanto para el conjunto de validación, como para el conjunto de test. (5 pts)